In [2]:
!pip install --no-cache-dir -q transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 146.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 205.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 148.2 MB/s eta 0:00:00


In [22]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda() if torch.cuda.is_available() else None
model.eval()

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

### test untrained model

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

pred_df = pd.read_csv('/content/nvidia_stt_ru_conformer_ctc_large.csv', sep='\t')
output_real = []
output_pred = []

for text_real, text_pred in tqdm(list(zip(pred_df['real'], pred_df['pred']))):
    output_real.append(summarize(text_real, n_words=20))
    output_pred.append(summarize(text_pred, n_words=20))

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
output_real

['Скажите пожалуйста свой пароль от почты, чтобы выдать вам новый пароль.',
 'Скажите пожалуйста свой пароль от почты, чтобы выдать вам новый пароль.',
 'Скажите пожалуйста свой пароль от почты, чтобы выдать вам новый пароль.',
 'Скажите пожалуйста свой пароль от почты, чтобы выдать вам новый пароль.',
 'Скажите пожалуйста свой пароль от почты, чтобы выдать вам новый пароль.',
 'техническая поддержка спбгу, слушаю вас все до свидания.',
 'техническая поддержка спбгу, слушаю вас все до свидания.',
 'техническая поддержка спбгу, слушаю вас все до свидания.',
 'техническая поддержка спбгу, слушаю вас все до свидания.',
 'техническая поддержка спбгу, слушаю вас все до свидания.',
 'Скажите пожалуйста номер вашего общежития, который отрубает интернет.',
 'Скажите пожалуйста номер вашего общежития, который отрубает интернет.',
 'Скажите пожалуйста номер вашего общежития, который отрубает интернет.',
 'Скажите пожалуйста номер вашего общежития, который отрубает интернет.',
 'Скажите пожалуйст

In [ ]:
output_pred

['Спасибо за вашу поддержку.',
 'Скажите пожалуйста, какой факультет у вас?',
 'Хорошо, что вы зашли в деканат показать студентский билет.',
 'Надеюсь, что вы не сможете зайти в деканат показать вам студенческий билет.',
 'Спасибо за поддержку. Заходите в деканат показать стунческий билет, чтобы выдать вам новый пароль.',
 'Надеюсь, что у вас в аудитории включено оборудование.',
 'Спасибо за поддержку.',
 'Надеюсь, что у вас включено оборудование в аудитории.',
 'Хорошо слушаю васра.',
 'Спасибо за поддержку.',
 'Скажите пожалуйста номер вашего общежития, пятнадцатая общага семьсот девять.',
 'Скажите пожалуйста номер вашего общежития, пятнадцатая общага семьсот девять.',
 'Скажите пожалуйста номер вашего общежития.',
 'Скажите пожалуйста номер вашего чяжите доко пятнадцатого общага семьсот девять.',
 'Скажите пожалуйста номер вашего общежития.',
 'Хорошо назовите свой номер абонента.',
 'У меня на компьютере нет подключения к интернету.',
 'Хорошо назовите свой номер абонента, который

### word dropout

In [ ]:
import nltk
nltk.download('popular')

In [26]:
import random

def apply_word_dropout(texts, dropout_prob):
    """
    Apply word dropout by replacing on the <unk> token

    Params:
    -------
    texts (list): list of tokenized texts
    dropout_prob (float): probability of replacing word on <unk> token

    Return:
    -------
    processed_texts (list) - list of tokenised and preprocessed texts
    """

    processed_texts = []
    for text in texts:
        result = []
        tokens = nltk.word_tokenize(text)
        for token in tokens:
            if random.random() < dropout_prob:
                result.append("<unk>")
            else:
                result.append(token)
        processed_texts.append(' '.join(result))
    return processed_texts

In [ ]:
!pip install -q datasets

In [21]:
from datasets import Dataset
import pandas as pd

pred_df = pd.read_csv('/content/nvidia_stt_ru_conformer_ctc_large.csv', sep='\t')
dataset = Dataset.from_pandas(pred_df)
dataset

Dataset({
    features: ['real', 'pred'],
    num_rows: 30
})

In [51]:
prefix = "summarize: "
dropout_prob = 0.2

def preprocess_function(examples):

    dropped_inputs = apply_word_dropout(examples["real"], dropout_prob)
    
    inputs = [prefix + doc for doc in dropped_inputs]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # labels = tokenizer(text_target=examples["summa"], max_length=128, 
    #                    truncation=True)
    # 
    # model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [42]:
apply_word_dropout(list(pred_df['real']), 0.3)[0]

'добрый день техническая поддержка <unk> <unk> вас добрый <unk> я <unk> к <unk> свой <unk> от почты <unk> мне нужно сделать <unk> пожалуйста свои <unk> имя отчество филатова мария евгеньевна <unk> какой у вас факультет матмех какой <unk> <unk> номер группы 20 б 07 мм <unk> хорошо с <unk> <unk> можете зайти <unk> <unk> <unk> студенческий билет и вам выдадут новый пароль а дистанционно никак не <unk> <unk> <unk> <unk> но <unk> не можем <unk> удостоверения <unk> выдать вам новый <unk> ладно <unk> спасибо <unk> свидания до свидания'

In [52]:
tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [53]:
tokenized

Dataset({
    features: ['real', 'pred', 'input_ids', 'attention_mask'],
    num_rows: 30
})

In [ ]:
tokenized['input_ids'][0]